In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import time
import re

In [ ]:
from sklearn.svm import SVC

In [ ]:
import sklearn.svm

In [ ]:
import xgboost as xgb

In [ ]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.5 MB/s eta 0:00:00


In [ ]:
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [ ]:
filename = "/content/drive/My Drive/Datasets/e_2k.csv"

df = pd.read_csv(filename,
                 encoding="utf-8", encoding_errors="replace")
# replace values
df.replace({'Not Environmental': 0, 'Environmental': 1}, inplace=True)

# Data Preprocessing

In this section we will prepare the dataset in order to use in SVM model. Before starting I want to explain processes with 1 line explanations



*   Cleaning Everything Except Alphabetical and Numerical Characters: We'll clean unrelevant digits using regular expressions
*   Tokenizing and Lemmatizing: We'll split texts into their words and convert words to their base form (dogs=>dog)
*   Removing Stopwords: We'll remove words that have no special meaning (such as the,will,was)
*   Joining Prepared Strings: We'll join the words
*   Bag of Words Approach: We'll create a bag of word. In bag of words approach each feature shows whether the text contains the word or not.







In [ ]:
cleanedData = []

lemma = WordNetLemmatizer()
swords = stopwords.words("english")
for text in df["text"]:

    # Cleaning everything except alphabetical and numerical characters
    text = re.sub("[^a-zA-Z0-9]"," ",text)

    # Tokenizing and lemmatizing
    text = nltk.word_tokenize(text.lower())
    text = [lemma.lemmatize(word) for word in text]

    # Removing stopwords
    text = [word for word in text if word not in swords]

    # Joining
    text = " ".join(text)

    cleanedData.append(text)

In [ ]:
# check if the data is cleaned
for i in range(0,5):
    print(cleanedData[i],end="\n\n")

balance included kind donation prepared accordance b4si guideline

internal control risk management board audit committee responsible ensuring suitable internal control system prevent detect fraud error designed implemented third party service provider company also responsible reviewing effectiveness control

analysis doe take account energy mix related electricity consumption ttv portfolio issuer

change chief financial officer plc board chair position clare bousfield stepped board effect 1 october 2021 transitioned role managing director retail saving business clare continued receive salary benefit reflective new role

consideration prioritisation climate risk also built decision making governance process requirement key strategic board risk assessment paper



Now we can create our bag of words

In [ ]:
vectorizer = CountVectorizer(max_features=10000)
BOW = vectorizer.fit_transform(cleanedData)

we can split our set into train and test.

In [ ]:
x_train = list()
x_test = list()
y_train = list()
y_test = list()
for label in [0, 1]:
  x_train,x_test,y_train,y_test = train_test_split(BOW, np.asarray(df["label"]),
                                                  train_size=250,
                                                  test_size=250,
                                                  random_state=42)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(250, 4937)
(250, 4937)
(250,)
(250,)


Support Vector Machine Classifier Modeling

In [ ]:
start_time = time.time()

model = SVC()
model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting SVC took {} seconds".format(process_time))

Fitting SVC took 0.02 seconds


In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 72.39999999999999%


# Hyperparameter tuning using Optuna

In [ ]:
clf = SVC(gamma="auto")
param_distributions = {
        "C": optuna.distributions.FloatDistribution(1e-10, 1e10, log=True),
        "degree": optuna.distributions.IntDistribution(1, 5),
    }

optuna_search = optuna.integration.OptunaSearchCV(
        clf, param_distributions, n_trials=100, timeout=600, verbose=2
    )


optuna_search.fit(x_train,y_train)

print("Best trial:")
trial = optuna_search.study_.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

<ipython-input-28-4062d26f2c32>:7: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
[I 2024-04-30 02:31:23,024] A new study created in memory with name: no-name-a8c5a0df-a9e3-4115-8c72-30dfbd58f839
INFO:optuna_integration.sklearn:Searching the best hyperparameters using 250 samples...
[I 2024-04-30 02:31:23,085] Trial 0 finished with value: 0.692 and parameters: {'C': 0.23956925449868097, 'degree': 4}. Best is trial 0 with value: 0.692.
[I 2024-04-30 02:31:23,132] Trial 1 finished with value: 0.692 and parameters: {'C': 2.7250314754066677e-08, 'degree': 4}. Best is trial 0 with value: 0.692.
[I 2024-04-30 02:31:23,180] Trial 2 finished with value: 0.692 and parameters: {'C': 2.262025213088064e-10, 'degree': 3}. Best is trial 0 with value: 0.692.
[I 2024-04-30 02:31:23,229] Trial 3 finished with value: 0.692 and parameters: {'C': 9.080130353084631e-10, 'degree': 2}. 

Best trial:
  Value:  0.8080000000000002
  Params: 
    C: 1102.981824482664
    degree: 4


In [ ]:
trial.params

{'C': 1102.981824482664, 'degree': 4}

In [ ]:
start_time = time.time()

model = SVC(gamma="auto", degree=trial.params['degree'], C=trial.params['C'])
model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting SVC took {} seconds".format(process_time))

Fitting SVC took 0.01 seconds


In [ ]:
trial.params['degree']

4

In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 83.2%


In [ ]:
def evaluate(y_true, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
evaluate(y_true=y_test, y_pred=predictions)

Accuracy: 0.832
Accuracy for label 0: 0.928
Accuracy for label 1: 0.639

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.93      0.88       167
           1       0.82      0.64      0.72        83

    accuracy                           0.83       250
   macro avg       0.83      0.78      0.80       250
weighted avg       0.83      0.83      0.83       250


Confusion Matrix:
[[155  12]
 [ 30  53]]


# build model for XGBoost

In [ ]:
start_time = time.time()

xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting XGBoost took {} seconds".format(process_time))

Fitting XGBoost took 0.2 seconds


In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 83.2%


# Hyperparameter tuning using Optuna

In [ ]:
def objective(trial):
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dvalid = xgb.DMatrix(x_test, label=y_test)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test,predictions)
    return accuracy

In [ ]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2024-04-30 02:54:13,106] A new study created in memory with name: no-name-db83e704-c1c4-41c5-af3c-f1ec8233c31e


[0]	validation-auc:0.75269
[1]	validation-auc:0.78840
[2]	validation-auc:0.78840
[3]	validation-auc:0.78840
[4]	validation-auc:0.78840
[5]	validation-auc:0.78840
[6]	validation-auc:0.78840
[7]	validation-auc:0.78840
[8]	validation-auc:0.78840
[9]	validation-auc:0.78840


[I 2024-04-30 02:54:13,165] Trial 0 finished with value: 0.832 and parameters: {'booster': 'gbtree', 'lambda': 2.32750354564272e-07, 'alpha': 9.923228968864207e-05, 'max_depth': 6, 'eta': 1.7331098296958403e-06, 'gamma': 0.00011838497022715292, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.832.


[0]	validation-auc:0.65966
[1]	validation-auc:0.65966
[2]	validation-auc:0.65966
[3]	validation-auc:0.65966
[4]	validation-auc:0.65966
[5]	validation-auc:0.65966
[6]	validation-auc:0.65966
[7]	validation-auc:0.65966
[8]	validation-auc:0.65966
[9]	validation-auc:0.65966


[I 2024-04-30 02:54:13,209] Trial 1 finished with value: 0.832 and parameters: {'booster': 'gbtree', 'lambda': 1.4934532585985831e-05, 'alpha': 1.1802014024753904e-06, 'max_depth': 2, 'eta': 0.0003051574264155878, 'gamma': 0.0020589406722531013, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.832.


...

[0]	validation-auc:0.83143
[1]	validation-auc:0.83143
[2]	validation-auc:0.83143
[3]	validation-auc:0.83143
[4]	validation-auc:0.83143
[5]	validation-auc:0.83143


[I 2024-04-30 02:54:20,752] Trial 98 pruned. Trial was pruned at iteration 5.


[0]	validation-auc:0.67614
[1]	validation-auc:0.71211
[2]	validation-auc:0.71189
[3]	validation-auc:0.71214
[4]	validation-auc:0.71193
[5]	validation-auc:0.71218


[I 2024-04-30 02:54:20,820] Trial 99 pruned. Trial was pruned at iteration 5.


FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.832], datetime_start=datetime.datetime(2024, 4, 30, 2, 54, 13, 108129), datetime_complete=datetime.datetime(2024, 4, 30, 2, 54, 13, 164551), params={'booster': 'gbtree', 'lambda': 2.32750354564272e-07, 'alpha': 9.923228968864207e-05, 'max_depth': 6, 'eta': 1.7331098296958403e-06, 'gamma': 0.00011838497022715292, 'grow_policy': 'depthwise'}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.7526873962917539, 1: 0.7883991054036505, 2: 0.7883991054036505, 3: 0.7883991054036505, 4: 0.7883991054036505, 5: 0.7883991054036505, 6: 0.7883991054036505, 7: 0.7883991054036505, 8: 0.7883991054036505, 9: 0.7883991054036505}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'alpha': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'max_depth': IntDistribution(high=9, log=False, low=1, step=1), 'eta': FloatDistri

In [ ]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

Best trial:
  Value:  0.832
  Params: 
    booster: gbtree
    lambda: 2.32750354564272e-07
    alpha: 9.923228968864207e-05
    max_depth: 6
    eta: 1.7331098296958403e-06
    gamma: 0.00011838497022715292
    grow_policy: depthwise


In [ ]:
start_time = time.time()

xgb_model = xgb.XGBClassifier(**trial.params)
xgb_model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting XGBoost took {} seconds".format(process_time))

Fitting XGBoost took 1.84 seconds


In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 83.2%


In [ ]:
evaluate(y_true=y_test, y_pred=predictions)

Accuracy: 0.832
Accuracy for label 0: 0.928
Accuracy for label 1: 0.639

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.93      0.88       167
           1       0.82      0.64      0.72        83

    accuracy                           0.83       250
   macro avg       0.83      0.78      0.80       250
weighted avg       0.83      0.83      0.83       250


Confusion Matrix:
[[155  12]
 [ 30  53]]
